In [1]:
import pandas as pd

from notebooks.ford.decode import search, print_breakdown


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200)

df_nhtsa = search(
  # searches=['Bronco Sport', 'Escape'],
  # searches=['Explorer'],
  # searches=['F-150'],
  min_model_year=2018,  # one year before the first supported
  include_openpilot=True,
  include_police=False,
  skip_missing_asbuilt=True,
)

print()
print_breakdown(df_nhtsa)

Loaded 5535 VINs (filter_comment=None, include_openpilot=True, skipped=131, missing_asbuilt=0)


Decoding VINs: 100%|██████████| 5535/5535 [00:00<00:00, 18783.03it/s]


Decoded 5535 VINs

Model
Aviator            178
Bronco              87
Bronco Sport       372
Continental          4
Corsair            173
Ecosport            74
Edge               474
Escape             786
Expedition          71
Expedition MAX      76
Explorer           845
F-150              668
F-250               74
F-350               43
F-450                5
F-550                1
Fiesta              55
Flex                18
Focus               48
Fusion              67
GT                   3
MKC                  8
MKT                  1
MKZ                 37
Maverick           332
Mustang            208
Mustang Mach-E     230
Nautilus           131
Navigator           79
Navigator L         64
Ranger              45
Taurus              14
Transit            185
Transit Connect     51
dtype: int64

Model            ModelYear
Aviator          2020          40
                 2021          14
                 2022          16
                 2023          78
                

In [2]:
SKIP = [
  'AirBagLocFront',
  'AirBagLocKnee',
  'AirBagLocSide',
  'BodyClass',
  'DisplacementCC',
  'DisplacementCI',
  'DisplacementL',
  'EngineCylinders',
  'GVWR',
  'LowerBeamHeadlampLightSource',
  'Make',
  'MakeID',
  'Manufacturer',
  'ManufacturerId',
  'ModelID',
  'NCSABodyType',
  'NCSAMake',
  'NCSAModel',
  'PlantCity',
  'PlantCompanyName',
  'PlantCountry',
  'PlantState',
  'Seats',
  'WheelSizeFront',
  'WheelSizeRear',
  'VehicleType',
  'VIN',
  'VehicleDescriptor',
  'WheelSizeFront',
  'WheelSizeRear',
]

KEEP = [
  # 'DisplacementL',
  'DriveType',
  # 'ElectrificationLevel',
  # 'FuelTypePrimary',
  # 'FuelTypeSecondary',
  'Model',
  'Series',
  'Trim',
]

properties = {}

for col in KEEP:
  if col not in df_nhtsa.columns:
    print(f'WARNING: {col} not in df_nhtsa.columns')

for col in df_nhtsa.columns:
  if col in SKIP:
    continue

  property_values = set(df_nhtsa[col].unique())
  if col not in KEEP:
    if '' in property_values:
      continue
    if len(property_values) == 1:
      continue

  properties[col] = property_values

df_nhtsa.drop(columns=['FuelTypePrimary', 'FuelTypeSecondary'], inplace=True, errors='ignore')
properties.pop('FuelTypePrimary', None)
properties.pop('FuelTypeSecondary', None)

properties

{'DriveType': {'2WD', '4WD', 'AWD', 'FWD', 'RWD', 'Unknown'},
 'ElectrificationLevel': {'BEV', 'FHEV', 'HEV', 'ICE', 'PHEV'},
 'Model': {'Aviator',
  'Bronco',
  'Bronco Sport',
  'Continental',
  'Corsair',
  'Ecosport',
  'Edge',
  'Escape',
  'Expedition',
  'Expedition MAX',
  'Explorer',
  'F-150',
  'F-250',
  'F-350',
  'F-450',
  'F-550',
  'Fiesta',
  'Flex',
  'Focus',
  'Fusion',
  'GT',
  'MKC',
  'MKT',
  'MKZ',
  'Maverick',
  'Mustang',
  'Mustang Mach-E',
  'Nautilus',
  'Navigator',
  'Navigator L',
  'Ranger',
  'Taurus',
  'Transit',
  'Transit Connect'},
 'ModelYear': {2018, 2019, 2020, 2021, 2022, 2023, 2024},
 'Series': {'',
  '150',
  '250',
  '350',
  '350 HD',
  'AWD MKZ Reserve I',
  'AWD MKZ Reserve II',
  'Active',
  'BADLANDS',
  'BASE',
  'BIG BEND',
  'BL',
  'Badlands',
  'Base',
  'Base, Big Bend, Black Diamond, Outer Banks',
  'Base, Big Bend, Black Diamond, Outer Banks, Badlands, Wildtrak',
  'Base, Big Bend, Black Diamond, Outer Banks, Badlands, Wild

### Combine NHTSA and Ford AsBuilt Data

We fetch the factory part numbers (software and hardware) from the Ford AsBuilt data and combine it with the NHTSA data.

In [3]:
from panda.python.uds import DATA_IDENTIFIER_TYPE

from notebooks.ford.asbuilt import AsBuiltData
from notebooks.ford.ecu import FordEcu
from notebooks.ford.settings import VehicleSetting, VehicleSettings


df_fw = df_nhtsa.copy()

# Drop columns that we don't care about (not in the properties)
df_fw.drop(
  columns=[col for col in df_nhtsa.columns if col not in properties and col != 'VIN'],
  inplace=True,
)


ecus = {
  'abs': FordEcu.AntiLockBrakeSystem,
  # 'engine': FordEcu.PowertrainControlModule,
  'eps': FordEcu.PowerSteeringControlModule,
  'fwdCamera': FordEcu.ImageProcessingModuleA,
  'fwdRadar': FordEcu.CruiseControlModule,
}


def get_ecu_identifier(ecu: FordEcu, identifier: int):
  def apply(row):
    data = AsBuiltData.from_vin(row['VIN'])
    if ecu not in data.ecus:
      return ''
    return data.get_identifier(ecu, identifier)
  return apply


def get_setting(settings: list[VehicleSetting]):
  def apply(row):
    # FIXME: assumes same ecu for each setting
    for setting in settings:
      ecu = setting.ecu
      if type(ecu) is tuple:
        ecu, pn_core = ecu
      else:
        pn_core = None

      ecu_name = next(filter(lambda name: ecus[name] == ecu, ecus.keys()))
      ecu_part = row[f'{ecu_name}_part']
      if not ecu_part:
        return 'Missing ECU'

      data = AsBuiltData.from_vin(row['VIN'])
      if ecu not in data.ecus:
        return 'Missing ECU'

      core = data.get_identifier(ecu, 0xF111).split('-')[1]
      if pn_core and core != pn_core:
        # return 'Wrong ECU'
        continue

      return data.get_setting_value(setting)
    return 'Not supported'
  return apply


# Add the ECU identifiers
for name, ecu in ecus.items():
  df_fw[f'{name}_fw'] = df_fw.apply(
    get_ecu_identifier(
      ecu, DATA_IDENTIFIER_TYPE.VEHICLE_MANUFACTURER_ECU_SOFTWARE_NUMBER
    ),
    axis=1,
  )
  df_fw[f'{name}_part'] = df_fw.apply(get_ecu_identifier(ecu, 0xF111), axis=1)


# Determine platform (pre-Q3, Q3 or Q4)
def get_platform(row):
  camera_pn = row['fwdCamera_part']
  core = camera_pn.split('-')[1] if camera_pn else None
  if not core:
    return 'No camera'

  platform = {
    '14F403': 'Q3',
    '14G025': 'pre-Q3',  # guess, seen on 2020 Fusion/Mondeo
    '14H107': 'Q4',
  }.get(core)

  if platform:
    return platform
  assert False, f'Unhandled platform for {row["ModelYear"]} {row["Model"]} {camera_pn=}'


df_fw['Platform'] = df_fw.apply(get_platform, axis=1)


# Add settings
settings = {
  'ACC (IPMA)': [VehicleSettings.ipma_enable_adaptive_cruise, VehicleSettings.ipma_vehicle_cfg_acc_type],
  'TJA (IPMA)': [VehicleSettings.ipma_enable_traffic_jam_assist, VehicleSettings.ipma_module_feature_cfg_tja],
  'HwyAssist (IPMA)': [VehicleSettings.ipma_module_feature_cfg_hwy_assist],
}
for name, setting in settings.items():
  df_fw[name] = df_fw.apply(get_setting(setting), axis=1)


# Drop columns that we don't care about (not in the properties)
df_fw.drop(
  columns=[col for col in df_nhtsa.columns if col not in properties and col != 'VIN'],
  inplace=True,
  errors='ignore',
)


# Merge 'Series' and 'Trim' properties (they should be mutually exclusive)
if 'Trim' in properties and 'Series' in properties:
  def get_series_or_trim(row):
    series, trim = row['Series'], row['Trim']
    if series == 'F-Series':
      return trim
    elif series and trim:
      return f'{series} ({trim})'
      # raise ValueError(f'{row["VIN"]} Both Series and Trim are set: {series} and {trim}')
    return series or trim

  df_fw['Trim'] = df_fw.apply(get_series_or_trim, axis=1)
  df_fw.drop(columns=['Series'], inplace=True)
  properties.pop('Series', None)
elif 'Series' in properties:
  df_fw.rename(columns={'Series': 'Trim'}, inplace=True)
  properties['Trim'] = properties.pop('Series')


# Drop columns that are all empty
df_fw = df_fw.loc[:, (df_fw != '').any(axis=0)]


# Drop the VIN
df_fw.drop(columns=['VIN'], inplace=True, errors='ignore')


# Sort by columns
main_columns = []
if 'ModelYear' in df_fw.columns:
  main_columns.append('ModelYear')
if 'Series' in df_fw.columns:
  main_columns.append('Series')

extra_columns = list(set(properties.keys()) - set(main_columns))

# + [f'code_{name}' for name in settings.keys()] 
df_fw.sort_values(
  by=main_columns + extra_columns + [f'{name}_fw' for name in ecus.keys()],
  ascending=False,
  inplace=True,
  ignore_index=True,
)

# Delete 'Series' column
df_fw.drop(columns=['Series'], inplace=True, errors='ignore')


# Drop columns that are all the same
# df_fw = df_fw.loc[:, df_fw.apply(pd.Series.nunique) != 1]


# Add asterisks to column name where the value is the same for all rows
# df_fw.rename(
#   columns={
#     col: f'*{col}' if len(set(df_fw[col].unique())) == 1 else col
#     for col in df_fw.columns
#   },
#   inplace=True,
# )

# Remove duplicate rows

count = len(df_fw)
df_fw.drop_duplicates(inplace=True)
print(f'Removed {count - len(df_fw)} duplicate rows')


df_fw

Unknown ECU address: 1808 (0x710)
0x710 {61706: 'MB3C-14E098-BD', 61712: 'DSMB3C-14E090-AF', 61713: 'MB3C-14E099-AD', 61715: 'MB3C-14E090-AF', 61832: 'MB3C-14E093-AG', 61836: '0000000001879289'}
Unknown ECU address: 224 (0xe0)
0xe0 {61713: 'LC3A-14D607-BC', 61715: 'LC3A-12B533-BC', 61832: 'LC3A-14D609-BC', 61836: '0549117649051346'}
Unknown ECU address: 229 (0xe5)
0xe5 {61713: 'KC3A-5M208-BA', 61715: 'KC3A-5J242-EA', 61832: 'KC3A-5M203-AC', 61836: '2770200237102182'}
Unknown ECU address: 1808 (0x710)
0x710 {61706: 'MB3C-14E098-BD', 61712: 'DSMB3C-14E090-AF', 61713: 'MB3C-14E099-AD', 61715: 'MB3C-14E090-AF', 61832: 'MB3C-14E093-AG', 61836: '0000000001875267'}
Unknown ECU address: 1794 (0x702)
0x702 {61712: 'DSMU5T-14F548-BC', 61713: 'MU5T-14G009-JC', 61715: 'MU5T-14F548-JE', 61832: 'MU5T-14G003-JD', 61836: '3843'}
Unknown ECU address: 1891 (0x763)
0x763 {61712: 'DSMU5T-14F549-BC', 61713: 'MU5T-14G000-JC', 61715: 'MU5T-14F549-JE', 61832: 'MU5T-14F684-JD', 61836: '3919'}
Unknown ECU addre

,DriveType,ElectrificationLevel,Model,ModelYear,Trim,abs_fw,abs_part,eps_fw,eps_part,fwdCamera_fw,fwdCamera_part,fwdRadar_fw,fwdRadar_part,Platform,ACC (IPMA),TJA (IPMA),HwyAssist (IPMA)
0,Unknown,ICE,Mustang,2024,I4,PR3C-2D053-AJ,PR3C-14F065-AA,RL14-14D003-AA,PR3C-14F079-AD,RC5T-14H102-ABE,PR3T-14H107-BBE,,,Q4,Off,Disabled,Disabled
2,Unknown,ICE,Mustang,2024,I4,PR3C-2D053-AG,PR3C-14F065-AA,RL14-14D003-AA,PR3C-14F079-AD,RC5T-14H102-ABE,PR3T-14H107-BBE,RB5T-14D049-AB,ML3T-14F089-AH,Q4,RadarFusion,Enabled,Disabled
4,Unknown,ICE,Mustang,2024,I4,PR3C-2D053-AG,PR3C-14F065-AA,RL14-14D003-AA,PR3C-14F079-AD,RC5T-14H102-ABE,PR3T-14H107-BBE,,,Q4,Off,Disabled,Disabled
6,Unknown,ICE,Mustang,2024,I4,PR3C-2D053-AG,PR3C-14F065-AA,PR3C-14D003-AD,PR3C-14F079-AD,PC3T-14H102-ABR,PR3T-14H107-BBD,ML3T-14D049-AL,ML3T-14F089-AH,Q4,RadarFusion,Enabled,Disabled
7,Unknown,ICE,Mustang,2024,I4,PR3C-2D053-AG,PR3C-14F065-AA,PR3C-14D003-AD,PR3C-14F079-AD,PC3T-14H102-ABR,PR3T-14H107-BBE,ML3T-14D049-AL,ML3T-14F089-AH,Q4,RadarFusion,Enabled,Disabled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5501,4WD,ICE,F-150,2018,,JL34-2D053-AJ,JL34-14F065-AC,JL34-14D003-CA,JL34-14F079-AA,,,,,No camera,Missing ECU,Missing ECU,Missing ECU
5502,4WD,ICE,Expedition,2018,Platinum,JL14-2D053-AE,JL34-14F065-AC,JL14-14D003-CA,JR3C-14F079-AA,JL1T-14F397-AF,JL1T-14F403-CB,H1BT-14D049-AF,H1BT-14F089-AC,Q3,RadarFusion,Disabled,Not supported
5503,4WD,ICE,Ecosport,2018,Titanium - 4WD,GN15-2D053-CE,GN15-14F065-BE,GN15-14C217-AF,AE81-14F075-AA,,,,,No camera,Missing ECU,Missing ECU,Missing ECU
5504,2WD,ICE,Escape,2018,Titanium,GV61-14C036-CJ,GV61-14F067-CC,HV6T-14C217-AA,JV6T-14C262-AB,,,,,No camera,Missing ECU,Missing ECU,Missing ECU


In [4]:
df_fw_analysis = df_fw.copy()

SORTED_TRIM_LEVELS = [
  'Base',
  # Escape, Focus, Fusion
  'S',
  'SE',
  'SEL',
  'Titanium',
  'Active',
  'ST Line',
  'ST Line Select',
  'ST Line Elite',
  'ST Line Premium',
  'PHEV',
  # Bronco Sport
  'BIG BEND',
  'OUTER BANKS',
  'BADLANDS',
  'HERITAGE',
  # Explorer
  'XL',
  'XLT',
  'Limited',
  'ST',
  'Platinum',
  'Timberline',
  'King Ranch',
  # Aviator
  'Standard',
  'Reserve',
  'Livery',
  'Grand Touring',
]

def sort_trim_level(trim):
  if trim in SORTED_TRIM_LEVELS:
    return SORTED_TRIM_LEVELS.index(trim)
  return 1000

results = dict()

for ecu in reversed(ecus.keys()):
  print(f'# Ecu.{ecu}')

  fw_groups = df_fw_analysis \
    .drop(
      columns=[f'{name}_fw' for name in ecus.keys() if name != ecu] + [f'{name}_part' for name in ecus.keys() if name != ecu],
    ) \
    .rename(
      columns={
        f'{ecu}_fw': 'fw',
        f'{ecu}_part': 'part',
      },
    ) \
    .groupby(
      by=['part', 'fw'],
      dropna=False,
    )

  results[ecu] = fw_groups.agg(lambda x: ', '.join(map(lambda x: str(x), sorted(set(x)))))
  print(results[ecu].to_string())

  print()

with pd.ExcelWriter('ford_fw.xlsx', engine='xlsxwriter') as writer:
  for ecu, result in results.items():
    result.to_excel(writer, sheet_name=ecu)

# Ecu.fwdRadar
                                                      DriveType       ElectrificationLevel                                                                                                                                                                                                                                                 Model                                 ModelYear                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               